In [1]:
import cv2
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import math
import numpy as np
import os

In [2]:
AUDIO_RATE = 16000
y1, y2, y3, y4, x1, x2, x3, x4 = (128, 352, 128, 352, 68, 292, 428, 652)

In [3]:
import pims
video = pims.Video('Ses01F_impro01.avi')
frame = video[0:1]

In [4]:
def gen_video_array(vid):
    sess_id = vid[4]
    video_path = "E:/datasets/IEMOCAP_full_release.tar/IEMOCAP_full_release/IEMOCAP_full_release/Session{}/dialog/avi/DivX/{}.avi".format(sess_id,vid)
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    success=True
    frame_list = []
    while success:
        success, frame=cap.read()
        frame_list.append(frame)

    video_array = np.array(frame_list[:-1])
    del frame_list
    return video_array, fps

In [22]:
def save_arr2vid(arr, output_path, fps):
    print(arr.shape)
    out = cv2.VideoWriter(output_path, 
                          cv2.VideoWriter_fourcc(*'mp4v'), 
                          fps, (arr.shape[2], arr.shape[1]))
    for i in range(arr.shape[0]):
        data = arr[i, :, :, :]
        out.write(data)
    out.release()

In [37]:
def save_video_array(vid, output_dir, fps):
    sess_id = vid[4]
    video_path = "E:/datasets/IEMOCAP_full_release.tar/IEMOCAP_full_release/IEMOCAP_full_release/Session{}/dialog/avi/DivX/{}.avi".format(sess_id,vid)
    video = pims.Video(video_path)
    df = pd.read_excel('E:/datasets/preprocessed/extractionmap/cut_extractionmap{}.xlsx'.format(sess_id), sheet_name=vid)
    df["viframe"] = np.floor(df["iframe"]*fps/AUDIO_RATE).astype(int)
    df["vfframe"] = np.ceil(df["fframe"]*fps/AUDIO_RATE).astype(int)
    
    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    width = height = 88
    for index, row in df.iterrows():
        center = None
        varr = np.asarray(video[row["viframe"]:row["vfframe"]])
        if row["speaker"] == "L":
            arr = varr[:, 
                       y1:y2, 
                       x1:x2, :]
        elif row["speaker"] == "R":
            arr = varr[:, 
                       y3:y4, 
                       x3:x4, :]
            
        img_stack_sm = []
        for idx in range(arr.shape[0]):
            img = arr[idx, :, :, :]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray)
            
            if len(faces) != 0:
                x,y,w,h = faces[0]
                xc = int(round((2*x+w)/2))
                yc = int(round((2*y+h)/2))
                center = [xc, yc]
            else:
                if idx == 0:
                    center = [gray.shape[1]//2, gray.shape[0]//2]
                
            face = gray[center[1]-int(height/2):center[1]+int(height/2), 
                        center[0]-int(width/2):center[0]+int(width/2)]
            face = cv2.cvtColor(face, cv2.COLOR_GRAY2BGR)
            face = cv2.resize(face, (88, 88), interpolation=cv2.INTER_AREA)
            img_stack_sm.append(face)
            
        img_stack_sm = np.array(img_stack_sm)
        save_path = os.path.join(output_dir, "{}".format(row["smp_id"]))
        save_arr2vid(img_stack_sm, save_path+".mp4", fps)

### Pipeline

In [40]:
import torch
import sys
sys.path.append("E:/university/FYT/repos/multi_modal_ser")
from utils.dataset import MMSERDataset
mmser_ds = torch.load("E:/datasets/preprocessed/dataset/mmser_ds.pt")
df_ = mmser_ds.df_

In [41]:
df_["fn"] = df_["smp_id"].apply(lambda x: x[:-5])
smp_id_list = df_["fn"].unique().tolist()

In [42]:
smp_id_list = [i for i in smp_id_list if i.startswith("Ses01")]

In [43]:
# from tqdm import tqdm 

# fps=30
# for vid in tqdm(smp_id_list):
#     print(vid)
#     save_video_array(vid,
#                  "E:/datasets/preprocessed/face_roi/{}".format(vid.split("_")[0][:-1]),
#                  fps)

  0%|                                                                                                                           | 0/28 [00:00<?, ?it/s]

Ses01F_impro01
(60, 88, 88, 3)
(88, 88, 88, 3)
(42, 88, 88, 3)
(113, 88, 88, 3)
(95, 88, 88, 3)
(87, 88, 88, 3)
(46, 88, 88, 3)
(104, 88, 88, 3)
(137, 88, 88, 3)
(122, 88, 88, 3)
(86, 88, 88, 3)
(245, 88, 88, 3)
(140, 88, 88, 3)
(174, 88, 88, 3)
(169, 88, 88, 3)
(188, 88, 88, 3)
(81, 88, 88, 3)
(133, 88, 88, 3)
(124, 88, 88, 3)
(225, 88, 88, 3)
(89, 88, 88, 3)
(295, 88, 88, 3)
(134, 88, 88, 3)
(83, 88, 88, 3)
(218, 88, 88, 3)
(101, 88, 88, 3)
(86, 88, 88, 3)
(153, 88, 88, 3)
(182, 88, 88, 3)


  4%|████                                                                                                               | 1/28 [00:55<24:48, 55.13s/it]

(83, 88, 88, 3)
Ses01F_impro02
(280, 88, 88, 3)
(91, 88, 88, 3)
(89, 88, 88, 3)


  4%|████                                                                                                               | 1/28 [01:01<27:39, 61.45s/it]


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
# save_video_array(smp_id_list[0],
#                  "E:/datasets/preprocessed/face_roi/{}".format(smp_id_list[0].split("_")[0][:-1]),
#                  30)